### Import Libraries

In [13]:
import numpy as np
import pandas as pd
import datetime 
from datetime import timedelta
import re
import itertools

### Check valid date format through regular expressions

In [14]:
def valid_date(i):
    try:
        t = datetime.datetime.strptime(re.search(r'\d{2}(-|.){1}\d{2}(-|.)\d{4}',i).group(), '%d.%m.%Y')
        return [True,t]
    except:
        try:
            t = datetime.datetime.strptime(re.search(r'\d{2}(-|.){1}\d{2}(-|.)\d{4}',i).group(), '%d-%m-%Y')
            return [True,t]
        except:
            return [False]

### Handling Invalid data in date columns (check row wise if value matches the regular expression)

In [15]:
def check_rowwise_date(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = [valid_date(i)[1] if valid_date(i)[0] else None for i in df[col].values]

    df.dropna(axis = 1, how = 'all', inplace = True)
    df.ffill(axis = 0, inplace = True)
    df.bfill(axis = 0, inplace = True)
    print(df.dtypes)
    print(df.head(10))

### Keep only date columns

In [16]:
def check_date_col(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            try:
                df[col] = pd.to_datetime(df[col])
                #df[col] = pd.Timestamp(df[col])
            except ValueError:
                pass
            except TypeError:
                pass
        if df[col].dtype == 'datetime64[ns]':
            mask = df[col].values[0] < np.datetime64(datetime.datetime(1971, 12, 12))
            if mask:
                df.drop(columns = [col], axis=1, inplace=True)

    df.dropna(axis = 1, how = 'all', inplace = True)
    check_rowwise_date(df)

### Find Difference between each of the date columns

In [17]:
def difference_bet_date_col(df):
    df.columns = ['Date'+str(i) for i in df.columns]
    l = [i for i in itertools.combinations(df.columns, 2)]
    for i in l:
        df[i[0]+'-'+i[1]] = [(u-v).astype('timedelta64[D]')/ np.timedelta64(1, 'D') 
                            for u,v in zip(df[i[0]].values, df[i[1]].values)]

    print(df.head(10))

### Dummy Dataframe with different date formats, invalid data, and other datatypes

In [18]:
df = pd.DataFrame({1 : [1,'Pallavi',1141016266,'14/08/2020','11-08-2020','12.08.2020','13/08/2020 00:02:40'],
                   2:  [2,'Pratikshya',1141016267,'15-08-2020','12.08.2020','13/08/2020 00:02:40','10/08/2020'],
                   3:  [3,'Partha',1141016268,'16/08/2020','11-08-2020','13/08/2020 00:02:40','12.08.2020'],
                   4:  [4,'Pranati',1141016269,'17/08/2020','12.08.2020','13/08/2020 00:02:40','11-08-2020'],
                   5:  [5,'Parul',1141016261,'18/08/2020','11-08-2020','13/08/2020 00:02:40','12.08.2020'],
                   6:  [6,'Pakshi',1141016262,'19-08-2020','12.08.2020','13/08/2020 00:02:40','10/08/2020'],
                   7:  [7,'Paul',1141016263,'15/08/2020','11-08-2020','12.08.2020','13/08/2020 00:02:40'],
                   8:  [8,'Paulami',1141016264,'13/08/2020','sss','12.08.2020','13/08/2020 00:02:40'],
                   9:  [9,'Paras',1141016265,'10/08/2020','11-08-2020','12.08.2020','13/08/2020 00:02:40'],
                   10:  [10,'Panu',1141016296,'10/08/2020','11-08-2020','12.08.2020','March 01, 2019']})
df = df.T
df.head(10)

,0,1,2,3,4,5,6
1,1,Pallavi,1141016266,14/08/2020,11-08-2020,12.08.2020,13/08/2020 00:02:40
2,2,Pratikshya,1141016267,15-08-2020,12.08.2020,13/08/2020 00:02:40,10/08/2020
3,3,Partha,1141016268,16/08/2020,11-08-2020,13/08/2020 00:02:40,12.08.2020
4,4,Pranati,1141016269,17/08/2020,12.08.2020,13/08/2020 00:02:40,11-08-2020
5,5,Parul,1141016261,18/08/2020,11-08-2020,13/08/2020 00:02:40,12.08.2020
6,6,Pakshi,1141016262,19-08-2020,12.08.2020,13/08/2020 00:02:40,10/08/2020
7,7,Paul,1141016263,15/08/2020,11-08-2020,12.08.2020,13/08/2020 00:02:40
8,8,Paulami,1141016264,13/08/2020,sss,12.08.2020,13/08/2020 00:02:40
9,9,Paras,1141016265,10/08/2020,11-08-2020,12.08.2020,13/08/2020 00:02:40
10,10,Panu,1141016296,10/08/2020,11-08-2020,12.08.2020,"March 01, 2019"


In [19]:
# Checking which are date columns, discarding rest
check_date_col(df)

3    datetime64[ns]
4    datetime64[ns]
5    datetime64[ns]
6    datetime64[ns]
dtype: object
            3          4                   5                   6
1  2020-08-14 2020-08-11 2020-12-08 00:00:00 2020-08-13 00:02:40
2  2020-08-15 2020-08-12 2020-08-13 00:02:40 2020-10-08 00:00:00
3  2020-08-16 2020-08-11 2020-08-13 00:02:40 2020-12-08 00:00:00
4  2020-08-17 2020-08-12 2020-08-13 00:02:40 2020-11-08 00:00:00
5  2020-08-18 2020-08-11 2020-08-13 00:02:40 2020-12-08 00:00:00
6  2020-08-19 2020-08-12 2020-08-13 00:02:40 2020-10-08 00:00:00
7  2020-08-15 2020-08-11 2020-12-08 00:00:00 2020-08-13 00:02:40
8  2020-08-13 2020-08-11 2020-12-08 00:00:00 2020-08-13 00:02:40
9  2020-10-08 2020-08-11 2020-12-08 00:00:00 2020-08-13 00:02:40
10 2020-10-08 2020-08-11 2020-12-08 00:00:00 2019-03-01 00:00:00


In [20]:
# Finding the difference between each of the date columns
difference_bet_date_col(df)

        Date3      Date4               Date5               Date6  Date3-Date4  \
1  2020-08-14 2020-08-11 2020-12-08 00:00:00 2020-08-13 00:02:40          3.0   
2  2020-08-15 2020-08-12 2020-08-13 00:02:40 2020-10-08 00:00:00          3.0   
3  2020-08-16 2020-08-11 2020-08-13 00:02:40 2020-12-08 00:00:00          5.0   
4  2020-08-17 2020-08-12 2020-08-13 00:02:40 2020-11-08 00:00:00          5.0   
5  2020-08-18 2020-08-11 2020-08-13 00:02:40 2020-12-08 00:00:00          7.0   
6  2020-08-19 2020-08-12 2020-08-13 00:02:40 2020-10-08 00:00:00          7.0   
7  2020-08-15 2020-08-11 2020-12-08 00:00:00 2020-08-13 00:02:40          4.0   
8  2020-08-13 2020-08-11 2020-12-08 00:00:00 2020-08-13 00:02:40          2.0   
9  2020-10-08 2020-08-11 2020-12-08 00:00:00 2020-08-13 00:02:40         58.0   
10 2020-10-08 2020-08-11 2020-12-08 00:00:00 2019-03-01 00:00:00         58.0   

    Date3-Date5  Date3-Date6  Date4-Date5  Date4-Date6  Date5-Date6  
1        -116.0          0.0       -11